In [1]:
import os
import tensorflow as tf

In [2]:
tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./s_HanDB_Test.tfrecord"))
#f=open("./test_record.tfrecord",'rb')

In [3]:
def read_and_decode(filename_queue):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.string),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.float32)

    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size=10,
                                         capacity=30,
                                         num_threads=5,
                                         min_after_dequeue=10 )
    return images, labels

In [4]:
X = tf.placeholder(tf.float32, [None,64,64,1])
Y = tf.placeholder(tf.float32, [None, 2350])
keep_prob = tf.placeholder(tf.float32)

In [5]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
L1 = tf.nn.relu(L1)

L1 =  tf.nn.max_pool(L1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [6]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(L1,W2, strides=[1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)

L2 =  tf.nn.max_pool(L2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [7]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(L2,W3, strides=[1,1,1,1], padding = 'SAME')
L3 = tf.nn.relu(L3)

L3 =  tf.nn.max_pool(L3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [8]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(L3,W4, strides=[1,1,1,1], padding = 'SAME')
L4 = tf.nn.relu(L4)

L4 =  tf.nn.max_pool(L4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [9]:
W5 = tf.Variable(tf.random_normal([8*8*256, 1024], stddev=0.01))
L5 = tf.reshape(L4,[-1,8*8*256])
L5 = tf.matmul(L5, W5)
L5 = tf.nn.relu(L5)
L5 = tf.nn.dropout(L5,keep_prob)

In [10]:
W6 = tf.Variable(tf.random_normal([1024,2350],stddev=0.01))
logit = tf.matmul(L5, W6)

In [12]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

In [13]:
correct_prediction = tf.equal(tf.argmax(logit,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init)

    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    batch_size = 10

    for eopoch in range(100):
        image_batch, label_batch = read_and_decode(tf_record_filename_queue)

        for i in range(batch_size):
            sess.run([optimizer,loss],feed_dict={X: image_batch, Y:label_batch, keep_prob:0.5})
            train_acc=sess.run(accuracy, feed_dict={X:imabe_batch, Y:label_batch, keep_prob:1.0})
            print('step %d, training accuracy %g' % (i, train_acc))
        
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.UnimplementedError'>, Cast string to float is not supported
	 [[Node: Cast_1 = Cast[DstT=DT_FLOAT, SrcT=DT_STRING, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ParseSingleExample/Squeeze_label)]]


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.